In [1]:
import pandas as pd
import numpy as np
import chardet
import re
import difflib
import py_entitymatching as em

/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
with open('ltable.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large
    print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.7289493317667792, 'language': ''}


In [3]:
ltable = pd.read_csv('ltable.csv', encoding='ISO-8859-1')
ltable.head()

,ltable_id,title,category,brand,modelno,price
0,0,draper infrared remote transmitter,electronics - general,draper,121066,58.45
1,1,epson 1500 hours 200w uhe projector lamp elplp12,monitors,epson,elplp12,438.84
2,2,comprehensive two-piece 75 precision bnc jack for rg-59 set of 25,tv accessories,comprehensive,bj-2c7559,59.25
3,3,d-link dcs-1100 network camera,garden - general,d-link,dcs-1100,99.82
4,4,startech.com rkpw247015 24 outlet power strip,electronics - general,startech,rkpw247015,59.00


In [4]:
rtable = pd.read_csv('rtable.csv', encoding='ISO-8859-1')
rtable.head()

,rtable_id,title,category,brand,modelno,price
0,0,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,12.65
1,1,kodak black ink cartridge 10b 1163641,inkjet printer ink,kodak,1163641,10.28
2,2,kingston 128mx64 pc2700 compaq evo d320 ktc-d320 1g,computers accessories,kingston,ktc-d320 / 1g,33.75
3,3,kinamax ms-ues2 mini high precision usb 3-button 3d optical scroll wheel mouse with retractable ...,mice,kinamax,ms-ues2,6.99
4,4,kensington k72349us wireless mouse for netbooks,mice,kensington,k72349us,24.00


In [5]:
train_df = pd.read_csv('train.csv')
train_df.head()

,ltable_id,rtable_id,label,id
0,621,3167,0,1
1,2115,8789,0,2
2,278,9064,0,3
3,1673,13279,0,4
4,2305,2948,0,5


In [6]:
ltable['brand'].unique()

array(['draper', 'epson', 'comprehensive', 'd-link', 'startech', '3m',
       'hp', 'visioneer', 'brother', 'lacie', 'stanley', 'gear head',
       'avery', 'xfx', 'skb cases', 'innovera', 'mead', 'iogear',
       'quartet', 'aluratek', 'cyberpower', 'tribeca', nan, 'vantec',
       'fuji', 'da-lite', 'sunpak', 'tomtom', 'iluv', 'roocase',
       'peerless', 'iomega', 'kingston', 'sony', 'siig', 'read right',
       'dp video', 'targus', 'wilson jones', 'link depot', 'canon',
       'dreamgear', 'edge tech', 'sennheiser', 'samsung', 'kensington',
       'blackberry', 'arclyte', 'q-see', 'microsoft', 'lorex',
       'panasonic', 'cables to go', 'edge', 'case logic', 'sharpie',
       'vaultz', 'pelican storm', 'scosche', 'vizio', 'balt', 'dane-elec',
       'audiovox', 'verbatim', 'rca', 'encore software', 'bling software',
       'omnimount', 'koss', 'inland', 'imation', 'rubbermaid commercial',
       'xantech', 'pioneer', 'generic', 'v7', 'house of doolittle',
       'visual land', '

In [7]:
rtable['brand'].unique()

array(['koss', 'kodak', 'kingston', ..., 'goharddrive', 'powerdsine',
       'cimo'], dtype=object)

In [8]:
# Brand mapping dict
brand_match_dict = {}
rbrands = [str(x) for x in rtable['brand'].unique()]

for lbrand in ltable['brand'].unique():
    closest_brand = difflib.get_close_matches(str(lbrand), rbrands, n=1)
    if len(closest_brand) > 0:
        brand_match_dict[lbrand] = closest_brand[0]
    else:
        print(lbrand)
        brand_match_dict[lbrand] = ''
        

green onions supply
avf group


In [9]:
em.set_key(ltable, 'ltable_id')
em.set_key(rtable, 'rtable_id')

True

In [10]:
training_data = em.read_csv_metadata('train.csv', 
                         key='id',
                         ltable=ltable, rtable=rtable, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

train_validation = em.split_train_test(training_data, train_proportion=0.8, random_state=0)

Metadata file is not present in the given path; proceeding to read the csv file.


In [11]:
train_data = train_validation['train']
validataion_data = train_validation['test']
validataion_data.head()

,ltable_id,rtable_id,label,id
509,547,15262,0,742
2055,2502,16032,1,2913
5473,814,2813,0,7872
2337,1170,12137,0,3323
1235,2343,10785,0,1725


In [12]:
decision_tree_clf = em.DTMatcher(name='DecisionTree', random_state=0)
svm_clf = em.SVMMatcher(name='SVM', random_state=0)
rf_clf = em.RFMatcher(name='RF', random_state=0)
log_reg_clf = em.LogRegMatcher(name='LogReg', random_state=0)
lin_reg_clf = em.LinRegMatcher(name='LinReg')
nb_clf = em.NBMatcher(name='NB')

In [13]:
features = em.get_features_for_matching(ltable, rtable, validate_inferred_attr_types=False)
features.head()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,category_category_jac_qgm_3_qgm_3,category,category,qgm_3,qgm_3,jaccard,<function category_category_jac_qgm_3_qgm_3 at 0x1a23c4eae8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,category_category_cos_dlm_dc0_dlm_dc0,category,category,dlm_dc0,dlm_dc0,cosine,<function category_category_cos_dlm_dc0_dlm_dc0 at 0x1a25125d90>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,category_category_jac_dlm_dc0_dlm_dc0,category,category,dlm_dc0,dlm_dc0,jaccard,<function category_category_jac_dlm_dc0_dlm_dc0 at 0x1a25125e18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,category_category_mel,category,category,None,None,monge_elkan,<function category_category_mel at 0x1a25125d08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,category_category_lev_dist,category,category,None,None,lev_dist,<function category_category_lev_dist at 0x1a25125ea0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [14]:
train_vector = em.extract_feature_vecs(train_data, 
                            feature_table=features, 
                            attrs_after='label',
                            show_progress=True)

train_vector = em.impute_table(train_vector, 
                exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')
train_vector.head()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


,id,ltable_id,rtable_id,category_category_jac_qgm_3_qgm_3,category_category_cos_dlm_dc0_dlm_dc0,category_category_jac_dlm_dc0_dlm_dc0,category_category_mel,category_category_lev_dist,category_category_lev_sim,category_category_nmw,...,modelno_modelno_mel,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,label
2270,3227,1155,8963,0.000000,0.000000,0.000000,0.552381,18.000000,0.142857,-3.00000,...,0.525000,5.000000,0.166667,-1.000000,1.000000,0.0,0.806781,4.0,0.2,0
1921,2727,1987,8946,0.000000,0.000000,0.000000,0.460317,18.000000,0.142857,-2.00000,...,0.955556,1.000000,0.888889,8.000000,8.000000,0.0,0.718660,3.0,0.4,0
2501,3564,1649,1006,0.090909,0.000000,0.000000,0.644444,11.000000,0.266667,-6.00000,...,0.866667,3.000000,0.666667,6.000000,6.000000,0.0,0.500200,4.0,0.2,0
1843,2596,1686,19624,0.151071,0.142933,0.122830,0.628059,14.363386,0.309407,-1.68041,...,0.659571,6.223236,0.427959,1.851087,4.126375,0.0,0.651313,4.0,0.2,0
728,1036,547,7307,0.578947,0.816497,0.666667,0.865000,6.000000,0.625000,4.00000,...,0.520000,4.000000,0.200000,1.000000,2.000000,0.0,0.655477,3.0,0.4,0


In [15]:
clfs = [decision_tree_clf, svm_clf, rf_clf, log_reg_clf, lin_reg_clf, nb_clf]
result = em.select_matcher(clfs, table=train_vector, 
        exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'],
        k=10, # Num of fold
        target_attr='label', metric_to_select_matcher='f1', random_state=0)

In [16]:
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.610012,0.576658,0.590835
1,SVM,0.920498,0.319518,0.471548
2,RF,0.832119,0.508689,0.628341
3,LogReg,0.938968,0.464461,0.619138
4,LinReg,0.964889,0.449062,0.610322
5,NB,0.496939,0.562758,0.525429


In [17]:
final_clf = result['selected_matcher']
final_clf

In [25]:
rf_clf.fit(table=train_vector, 
       exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'], 
       target_attr='label')

In [26]:
log_reg_clf.fit(table=train_vector, 
       exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'], 
       target_attr='label')

In [30]:
lin_reg_clf.fit(table=train_vector, 
       exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'], 
       target_attr='label')

In [19]:
val_vector = em.extract_feature_vecs(validataion_data, feature_table=features, attrs_after='label', show_progress=False)
val_vector.head()

,id,ltable_id,rtable_id,category_category_jac_qgm_3_qgm_3,category_category_cos_dlm_dc0_dlm_dc0,category_category_jac_dlm_dc0_dlm_dc0,category_category_mel,category_category_lev_dist,category_category_lev_sim,category_category_nmw,...,modelno_modelno_mel,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,label
509,742,547,15262,0.578947,0.816497,0.666667,0.865000,6.0,0.625000,4.0,...,0.520000,4.0,0.200000,1.0,1.0,0.0,0.914356,3.0,0.400000,0
2055,2913,2502,16032,0.000000,0.000000,0.000000,0.495040,17.0,0.190476,-1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.724116,4.0,0.333333,1
5473,7872,814,2813,0.023810,0.000000,0.000000,0.671958,15.0,0.285714,1.0,...,0.000000,21.0,0.000000,-16.0,0.0,NaN,NaN,NaN,NaN,0
2337,3323,1170,12137,0.184211,0.408248,0.250000,0.482287,15.0,0.347826,3.0,...,0.866667,1.0,0.750000,3.0,3.0,0.0,0.567635,4.0,0.200000,0
1235,1725,2343,10785,0.483871,0.707107,0.500000,0.696296,12.0,0.555556,3.0,...,0.857143,10.0,0.285714,-6.0,4.0,0.0,0.908712,3.0,0.500000,0


In [20]:
val_vector = em.extract_feature_vecs(val_vector, feature_table=features,
                            attrs_after='label', show_progress=False)

val_vector = em.impute_table(val_vector, 
                exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'],
                strategy='mean')

predictions = final_clf.predict(table=val_vector, exclude_attrs=['id', 'ltable_id', 'rtable_id', 'label'], 
              append=True, target_attr='predicted', inplace=False)

eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 84.51% (60/71)
Recall : 41.38% (60/145)
F1 : 55.56%
False positives : 11 (out of 71 positive predictions)
False negatives : 85 (out of 1359 negative predictions)


In [21]:
test_data = em.read_csv_metadata('test.csv', 
                         key='id',
                         ltable=ltable, rtable=rtable, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
test_data.head()

Metadata file is not present in the given path; proceeding to read the csv file.


,ltable_id,rtable_id,id
0,1853,2139,0
1,1718,11835,8
2,1624,5013,9
3,614,5969,10
4,1512,19684,15


In [22]:
test_vector = em.extract_feature_vecs(test_data, 
                            feature_table=features, 
                            show_progress=True)

test_vector = em.impute_table(test_vector, 
                exclude_attrs=['id', 'ltable_id', 'rtable_id'],
                strategy='mean')
test_vector.head()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


,id,ltable_id,rtable_id,category_category_jac_qgm_3_qgm_3,category_category_cos_dlm_dc0_dlm_dc0,category_category_jac_dlm_dc0_dlm_dc0,category_category_mel,category_category_lev_dist,category_category_lev_sim,category_category_nmw,...,modelno_modelno_jac_dlm_dc0_dlm_dc0,modelno_modelno_mel,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim
0,0,1853,2139,0.000000,0.000000,0.0,0.674603,11.0,0.266667,-1.0,...,0.000000,0.485965,16.000000,0.157895,-6.000000,1.000000,0.000000,0.000000,4.000000,0.200000
1,8,1718,11835,0.347826,0.707107,0.5,0.570175,11.0,0.421053,-3.0,...,0.083923,0.667792,6.309294,0.434100,1.866938,4.216861,0.000000,0.388506,5.000000,0.166667
2,9,1624,5013,0.000000,0.000000,0.0,0.505556,12.0,0.200000,-4.0,...,1.000000,1.000000,0.000000,1.000000,6.000000,6.000000,0.019976,0.608283,3.539752,0.336372
3,10,614,5969,0.068966,0.000000,0.0,0.730682,9.0,0.437500,2.0,...,0.000000,0.918981,1.000000,0.888889,8.000000,8.000000,0.000000,0.640180,3.000000,0.400000
4,15,1512,19684,0.023256,0.000000,0.0,0.613907,17.0,0.190476,1.0,...,0.083923,0.667792,6.309294,0.434100,1.866938,4.216861,0.000000,0.711879,4.000000,0.200000


In [28]:
rf_predictions = rf_clf.predict(table=test_vector, exclude_attrs=['id', 'ltable_id', 'rtable_id'], 
              append=True, target_attr='label', inplace=False)
rf_predictions.head()

,id,ltable_id,rtable_id,category_category_jac_qgm_3_qgm_3,category_category_cos_dlm_dc0_dlm_dc0,category_category_jac_dlm_dc0_dlm_dc0,category_category_mel,category_category_lev_dist,category_category_lev_sim,category_category_nmw,...,modelno_modelno_mel,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,label
0,0,1853,2139,0.000000,0.000000,0.0,0.674603,11.0,0.266667,-1.0,...,0.485965,16.000000,0.157895,-6.000000,1.000000,0.000000,0.000000,4.000000,0.200000,0
1,8,1718,11835,0.347826,0.707107,0.5,0.570175,11.0,0.421053,-3.0,...,0.667792,6.309294,0.434100,1.866938,4.216861,0.000000,0.388506,5.000000,0.166667,0
2,9,1624,5013,0.000000,0.000000,0.0,0.505556,12.0,0.200000,-4.0,...,1.000000,0.000000,1.000000,6.000000,6.000000,0.019976,0.608283,3.539752,0.336372,1
3,10,614,5969,0.068966,0.000000,0.0,0.730682,9.0,0.437500,2.0,...,0.918981,1.000000,0.888889,8.000000,8.000000,0.000000,0.640180,3.000000,0.400000,0
4,15,1512,19684,0.023256,0.000000,0.0,0.613907,17.0,0.190476,1.0,...,0.667792,6.309294,0.434100,1.866938,4.216861,0.000000,0.711879,4.000000,0.200000,0


In [29]:
log_reg_predictions = log_reg_clf.predict(table=test_vector, exclude_attrs=['id', 'ltable_id', 'rtable_id'], 
              append=True, target_attr='label', inplace=False)
log_reg_predictions.head()

,id,ltable_id,rtable_id,category_category_jac_qgm_3_qgm_3,category_category_cos_dlm_dc0_dlm_dc0,category_category_jac_dlm_dc0_dlm_dc0,category_category_mel,category_category_lev_dist,category_category_lev_sim,category_category_nmw,...,modelno_modelno_mel,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,label
0,0,1853,2139,0.000000,0.000000,0.0,0.674603,11.0,0.266667,-1.0,...,0.485965,16.000000,0.157895,-6.000000,1.000000,0.000000,0.000000,4.000000,0.200000,0
1,8,1718,11835,0.347826,0.707107,0.5,0.570175,11.0,0.421053,-3.0,...,0.667792,6.309294,0.434100,1.866938,4.216861,0.000000,0.388506,5.000000,0.166667,0
2,9,1624,5013,0.000000,0.000000,0.0,0.505556,12.0,0.200000,-4.0,...,1.000000,0.000000,1.000000,6.000000,6.000000,0.019976,0.608283,3.539752,0.336372,1
3,10,614,5969,0.068966,0.000000,0.0,0.730682,9.0,0.437500,2.0,...,0.918981,1.000000,0.888889,8.000000,8.000000,0.000000,0.640180,3.000000,0.400000,0
4,15,1512,19684,0.023256,0.000000,0.0,0.613907,17.0,0.190476,1.0,...,0.667792,6.309294,0.434100,1.866938,4.216861,0.000000,0.711879,4.000000,0.200000,0


In [31]:
lin_reg_predictions = lin_reg_clf.predict(table=test_vector, exclude_attrs=['id', 'ltable_id', 'rtable_id'], 
              append=True, target_attr='label', inplace=False)
lin_reg_predictions.head()

,id,ltable_id,rtable_id,category_category_jac_qgm_3_qgm_3,category_category_cos_dlm_dc0_dlm_dc0,category_category_jac_dlm_dc0_dlm_dc0,category_category_mel,category_category_lev_dist,category_category_lev_sim,category_category_nmw,...,modelno_modelno_mel,modelno_modelno_lev_dist,modelno_modelno_lev_sim,modelno_modelno_nmw,modelno_modelno_sw,price_price_exm,price_price_anm,price_price_lev_dist,price_price_lev_sim,label
0,0,1853,2139,0.000000,0.000000,0.0,0.674603,11.0,0.266667,-1.0,...,0.485965,16.000000,0.157895,-6.000000,1.000000,0.000000,0.000000,4.000000,0.200000,0
1,8,1718,11835,0.347826,0.707107,0.5,0.570175,11.0,0.421053,-3.0,...,0.667792,6.309294,0.434100,1.866938,4.216861,0.000000,0.388506,5.000000,0.166667,0
2,9,1624,5013,0.000000,0.000000,0.0,0.505556,12.0,0.200000,-4.0,...,1.000000,0.000000,1.000000,6.000000,6.000000,0.019976,0.608283,3.539752,0.336372,1
3,10,614,5969,0.068966,0.000000,0.0,0.730682,9.0,0.437500,2.0,...,0.918981,1.000000,0.888889,8.000000,8.000000,0.000000,0.640180,3.000000,0.400000,0
4,15,1512,19684,0.023256,0.000000,0.0,0.613907,17.0,0.190476,1.0,...,0.667792,6.309294,0.434100,1.866938,4.216861,0.000000,0.711879,4.000000,0.200000,0


In [32]:
ensemble_df = rf_predictions[['id', 'label']]
ensemble_df['label'] = ensemble_df['label'] + log_reg_predictions['label']
ensemble_df['label'] = ensemble_df['label'] + lin_reg_predictions['label']
ensemble_df.head()

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,label
0,0,0
1,8,0
2,9,3
3,10,0
4,15,0


In [33]:
ensemble_df['label'] = ensemble_df['label'].apply(lambda x : 1 if x > 1 else 0)
ensemble_df.head()

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,label
0,0,0
1,8,0
2,9,1
3,10,0
4,15,0


In [34]:
submission_df = ensemble_df[['id', 'label']]
submission_df.to_csv('submission.csv', index=False)